<a href="https://colab.research.google.com/github/kimdain0222/deep_learning2/blob/main/14%EC%A3%BC%EC%B0%A8(%EA%B5%90%EC%88%98ver)CNN_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Conv2D,Activation,MaxPooling2D,Flatten,Dense,BatchNormalization,Dropout
from tensorflow.keras.applications import ResNet50, VGG16

In [ ]:
dataset = np.load('vehicle_data.npz')
X = dataset['arr_0']
y = dataset['arr_1']

In [ ]:
X = X/255

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

- `ResNet50` 또는 `VGG16(weights,inlcude_top,input_shape)`
> - `weights`: 'imagenet'으로 설정한 경우, imagenet 데이터로 학습한 weight를 이용한다는 의미
> - `include_top`: False로 설정하는 경우, 분류부는 사용하지 않겠다는 의미
> - `input_shape`: 입력 image의 shape
- `ResNet50` 또는 `VGG16.trainable`: False로 설정하는 경우, weight를 학습하지 않고 그대로 사용하겠다는 의미

In [ ]:
transfer_layer = ResNet50(weights='imagenet',include_top=False,input_shape=(64,64,3))
transfer_layer.trainable = False

In [ ]:
vehicle_model = Sequential([
    # -----특징추출부는 ResNet50 사용
    transfer_layer,
    Flatten(),
    # -----분류부
    Dropout(0.2),
    Dense(units=128,activation='relu'),
    Dense(units=2,activation='softmax')
])

In [ ]:
vehicle_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2, 2, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,048,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,636,674 (93.98 MB)

 Trainable params: 1,048,962 (4.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
vehicle_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='best.weights.h5',
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             mode='min',
                             verbose=2)

In [ ]:
train_result = vehicle_model.fit(X_train,y_train,
                                 validation_split=0.3,
                                 epochs=30,
                                 batch_size=32,
                                 callbacks=[checkpoint])

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - acc: 0.6853 - loss: 0.5842
Epoch 1: val_loss improved from inf to 0.30870, saving model to best.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 542ms/step - acc: 0.6921 - loss: 0.5766 - val_acc: 0.9151 - val_loss: 0.3087
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - acc: 0.9024 - loss: 0.2900
Epoch 2: val_loss improved from 0.30870 to 0.22315, saving model to best.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 265ms/step - acc: 0.9033 - loss: 0.2855 - val_acc: 0.8774 - val_loss: 0.2231
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - acc: 0.9421 - loss: 0.1589
Epoch 3: val_loss improved from 0.22315 to 0.19068, saving model to best.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 247ms/step - acc: 0.9426 - loss: 0.1578 - val_acc: 0.9340 - val_loss: 0.1907
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - acc: 0.9426 - loss: 0.1556
Epoch 4: val_loss improved from 0.19068 to 0.18078, saving model to best.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 237ms/st

In [ ]:
plt.plot(train_result.history['loss'],label='training')
plt.plot(train_result.history['val_loss'],label='validation')
plt.legend()

- `X_test`를 이용한 성능 평가

In [ ]:
# y_test를 Label encoding함
y_test = le.transform(y_test)

In [ ]:
# ModelCheckpoint를 통해 저장된 최고 성능의 weight를 로드하지 않으면
# 학습이 종료된 시점의 모델을 사용함
vehicle_model.evaluate(X_test,y_test)

In [ ]:
# load_weights를 통해 최고 성능의 weight를 로드한 후, 성능 평가
vehicle_model.load_weights('best.weights.h5')
vehicle_model.evaluate(X_test,y_test)